In [ ]:
import numpy as np
import scipy, matplotlib.pyplot as plt, IPython.display as ipd
import librosa, librosa.display
import pandas as pd
import os
import util



In [ ]:
"""Load audio into python"""

i=0
# Please enter path of "genres" folder in "basepath" Eg. C:/GTZAN/genres/ Please ensure there is a "/" at the end.
basepath = 'path'
for folder in os.listdir(basepath):
    folder = basepath + folder
    for file in os.listdir(folder):
        locals()["F"+str(i)], sr = librosa.load(folder + '/' + file)
        print(i)
        i=i+1


In [ ]:
"""Define Function to determine key from Chroma vector. 
Source - https://gist.github.com/bmcfee/1f66825cef2eb34c839b42dddbad49fd"""

def ks_key(X):
    '''Estimate the key from a pitch class distribution
    
    Parameters
    ----------
    X : np.ndarray, shape=(12,)
        Pitch-class energy distribution.  Need not be normalized
        
    Returns
    -------
    major : np.ndarray, shape=(12,)
    minor : np.ndarray, shape=(12,)
    
        For each key (C:maj, ..., B:maj) and (C:min, ..., B:min),
        the correlation score for `X` against that key.
    '''
    X = scipy.stats.zscore(X)
    
    # Coefficients from Kumhansl and Schmuckler
    # as reported here: http://rnhart.net/articles/key-finding/
    major = np.asarray([6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88])
    major = scipy.stats.zscore(major)
    
    minor = np.asarray([6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17])
    minor = scipy.stats.zscore(minor)
    
    # Generate all rotations of major
    major = scipy.linalg.circulant(major)
    minor = scipy.linalg.circulant(minor)
    
    return major.T.dot(X), minor.T.dot(X)

In [ ]:
"""Calculate Chromagram arrays and summing along axis = 1 to get Chroma vector"""
Scales_predicted = ["C","C#","D","Eb","E","F","F#","G","Ab","A","Bb","B"]
Keys_predicted = []

for i in range (0,1000):
    print(i)
    chromagram = librosa.feature.chroma_cqt(locals()["F"+str(i)], sr=sr)
    summed_chromagram = np.sum((chromagram), axis = 1)


""" Calling key finding function (ks_key) to determine confidence score for every scale"""
    k = ks_key(summed_chromagram)
    k_pd = pd.DataFrame(k)
    largest_coordinate = k_pd.values.max()
    print(largest_coordinate)

"""Findind max confidence score and printing its corresponding scale"""

    for j in range (0,12):
        if k[0][j] == largest_coordinate:
           
            Keys_predicted.append(Scales_predicted[j] + " major")
        
        elif k[1][j] == largest_coordinate:

            Keys_predicted.append(Scales_predicted[j] + " minor")

print(Keys_predicted)

In [ ]:
"""Upload data of Actual Key"""

Scales_ref = ["A major","Bb major","B major","C major","C# major","D major","Eb major","E major","F major","F# major","G major","Ab major","A minor","Bb minor","B minor","C minor","C# minor","D minor","Eb minor","E minor","F minor","F# minor","G minor","Ab minor"]

Keys_actual=[]

i=0
# Please enter path of "keys/genres" folder in "basepath" Eg. C:/GTZAN//keys/genres/ Please ensure there is a "/" at the end.
basepath = 'path'
for folder in os.listdir(basepath):
    folder = basepath + folder
    for file in os.listdir(folder):
        Key_file = open(folder + '/' + file,"r")
        Key_file_data = Key_file.read()
        

        if Key_file_data != "-1":
            Keys_actual.append(Scales_ref[int(Key_file_data)])
        else:
            Keys_actual.append('Modulating')
        i=i+1
        Key_file.close()
        
print(Keys_actual)

In [ ]:
"""Define function to give weighted score to prediction
Full score given if estimated key is same as reference key.
50% score given if estimated key is perfect fifth above reference key.
25% score given if estimated key is relative major / minor of referenc key.
20% score given if estimated key is parallel major / minor of referenc key

Source - https://github.com/craffel/mir_eval/blob/master/mir_eval/key.py"""

KEY_TO_SEMITONE = {'c': 0, 'c#': 1, 'db': 1, 'd': 2, 'd#': 3, 'eb': 3, 'e': 4,
                   'f': 5, 'f#': 6, 'gb': 6, 'g': 7, 'g#': 8, 'ab': 8, 'a': 9,
                   'a#': 10, 'bb': 10, 'b': 11}


def validate_key(key):
    """Checks that a key is well-formatted, e.g. in the form ``'C# major'``.
    Parameters
    ----------
    key : str
        Key to verify
    """
    if len(key.split()) != 2:
        raise ValueError("'{}' is not in the form '(key) (mode)'".format(key))
    key, mode = key.split()
    if key.lower() not in KEY_TO_SEMITONE:
        raise ValueError(
            "Key {} is invalid; should be e.g. D or C# or Eb".format(key))
    if mode not in ['major', 'minor']:
        raise ValueError(
            "Mode '{}' is invalid; must be 'major' or 'minor'".format(mode))


def validate(reference_key, estimated_key):
    """Checks that the input annotations to a metric are valid key strings and
    throws helpful errors if not.
    Parameters
    ----------
    reference_key : str
        Reference key string.
    estimated_key : str
        Estimated key string.
    """
    for key in [reference_key, estimated_key]:
        validate_key(key)


def split_key_string(key):
    """Splits a key string (of the form, e.g. ``'C# major'``), into a tuple of
    ``(key, mode)`` where ``key`` is is an integer representing the semitone
    distance from C.
    Parameters
    ----------
    key : str
        String representing a key.
    Returns
    -------
    key : int
        Number of semitones above C.
    mode : str
        String representing the mode.
    """
    key, mode = key.split()
    return KEY_TO_SEMITONE[key.lower()], mode


def weighted_score(reference_key, estimated_key):
    """Computes a heuristic score which is weighted according to the
    relationship of the reference and estimated key, as follows:
    +------------------------------------------------------+-------+
    | Relationship                                         | Score |
    +------------------------------------------------------+-------+
    | Same key                                             | 1.0   |
    +------------------------------------------------------+-------+
    | Estimated key is a perfect fifth above reference key | 0.5   |
    +------------------------------------------------------+-------+
    | Relative major/minor                                 | 0.3   |
    +------------------------------------------------------+-------+
    | Parallel major/minor                                 | 0.2   |
    +------------------------------------------------------+-------+
    | Other                                                | 0.0   |
    +------------------------------------------------------+-------+
    Examples
    --------
    >>> ref_key = mir_eval.io.load_key('ref.txt')
    >>> est_key = mir_eval.io.load_key('est.txt')
    >>> score = mir_eval.key.weighted_score(ref_key, est_key)
    Parameters
    ----------
    reference_key : str
        Reference key string.
    estimated_key : str
        Estimated key string.
    Returns
    -------
    score : float
        Score representing how closely related the keys are.
    """
    validate(reference_key, estimated_key)
    reference_key, reference_mode = split_key_string(reference_key)
    estimated_key, estimated_mode = split_key_string(estimated_key)
    # If keys are the same, return 1.
    if reference_key == estimated_key and reference_mode == estimated_mode:
        return 1.
    # If keys are the same mode and a perfect fifth (differ by 7 semitones)
    if (estimated_mode == reference_mode and
            (estimated_key - reference_key) % 12 == 7):
        return 0.5
    # Estimated key is relative minor of reference key (9 semitones)
    if (estimated_mode != reference_mode == 'major' and
            (estimated_key - reference_key) % 12 == 9):
        return 0.3
    # Estimated key is relative major of reference key (3 semitones)
    if (estimated_mode != reference_mode == 'minor' and
            (estimated_key - reference_key) % 12 == 3):
        return 0.3
    # If keys are in different modes and parallel (same key name)
    if estimated_mode != reference_mode and reference_key == estimated_key:
        return 0.2
    # Otherwise return 0
    return 0.


def evaluate(reference_key, estimated_key, **kwargs):
    """Compute all metrics for the given reference and estimated annotations.
    Examples
    --------
    >>> ref_key = mir_eval.io.load_key('reference.txt')
    >>> est_key = mir_eval.io.load_key('estimated.txt')
    >>> scores = mir_eval.key.evaluate(ref_key, est_key)
    Parameters
    ----------
    ref_key : str
        Reference key string.
    est_key : str
        Estimated key string.
    kwargs
        Additional keyword arguments which will be passed to the
        appropriate metric or preprocessing functions.
    Returns
    -------
    scores : dict
        Dictionary of scores, where the key is the metric name (str) and
        the value is the (float) score achieved.
    """
    # Compute all metrics
    scores = {}

    scores['Weighted Score'] = util.filter_kwargs(
            weighted_score, reference_key, estimated_key)

    return scores


In [ ]:
"""Score calculation for the given model"""
Rating = []
for i in range (0,1000):
    if Keys_actual[i] == 'Modulating':
        pass
    else:
        r = evaluate (Keys_actual[int(i)],Keys_predicted[int(i)])
        Rating.append(r)
    

SumRating = 0
i = 0
for rating in Rating:
    Rating_ = int(rating['Weighted Score'])
    SumRating = SumRating + Rating_
    i = i+1
Accuracy = str((SumRating/i)*100)
print ("Accuracy of above model is " + Accuracy + " %")